In [1]:
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = "float16",
    bnb_4bit_use_double_quant = True,
    
)

c:\ProgramData\Anaconda3\envs\llm_final\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

model = './models/EXAONE-3.5-7.8B-Instruct/'
tokenizer = AutoTokenizer.from_pretrained(model)

# terminators = [
#     tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

In [37]:
llm = HuggingFacePipeline.from_model_id(
    model_id = model,
    task = "text-generation",
    pipeline_kwargs = dict(
        max_new_tokens = 2048,
        eos_token_id = tokenizer.eos_token_id,
        do_sample = False,
        #repetition_penalty = 1.2 # default 1.0, not response repeatly
        # temperature = 0.6,
        # top_p = 0.9
    ),
    device = 0,
    model_kwargs = {"trust_remote_code" : True, "quantization_config" : quantization_config},
    # model_kwargs = {"trust_remote_code" : True},
)

chat_model = ChatHuggingFace(llm = llm)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 7/7 [00:33<00:00,  4.84s/it]
Setting the `device` argument to None from 0 to avoid the error caused by attempting to move the model that was already loaded on the GPU using the Accelerate module to the same or another device.
Device set to use cuda:0


In [38]:
from langchain.memory import ConversationSummaryBufferMemory

memoryt = ConversationSummaryBufferMemory(
    llm = chat_model,
    max_token_limit = 120,
    return_messages = True,
)

In [39]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    AIMessage
)

In [12]:
messages = [
    SystemMessage(content = "You are EXAONE model from LG AI Research, a helpful assistant."),
    HumanMessage(
        content = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 철수의 연필 중 5개를 가져갔으면 각각 몇개의 연필을 가지고있는가?"
    )
]

In [13]:
ai_msg = chat_model.invoke(messages)

In [14]:
ai_msg

AIMessage(content='[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]\n[|user|]철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 철수의 연필 중 5개를 가져갔으면 각각 몇개의 연필을 가지고있는가?\n[|assistant|]철수가 처음에 20개의 연필을 가지고 있었습니다.\n\n1. **영희가 가져가는 경우:**\n   - 철수가 가진 연필의 절반을 영희가 가져갔으므로, 영희는 \\( \\frac{20}{2} = 10 \\)개의 연필을 가지게 됩니다.\n   - 이로 인해 철수는 \\( 20 - 10 = 10 \\)개의 연필을 남깁니다.\n\n2. **민수가 가져가는 경우:**\n   - 철수가 남은 연필은 10개입니다.\n   - 민수가 이 중 5개를 가져갔으므로, 민수는 5개의 연필을 가지게 됩니다.\n   - 철수는 남은 연필에서 민수가 가져간 연필을 뺀 나머지를 가지게 되므로, 철수는 \\( 10 - 5 = 5 \\)개의 연필을 남깁니다.\n\n따라서, 각 사람이 가지고 있는 연필의 수는 다음과 같습니다:\n- **영희:** 10개\n- **민수:** 5개', additional_kwargs={}, response_metadata={}, id='run-fa909462-7e48-4f6a-b840-d2facbe10641-0')

In [47]:
import pymupdf4llm

convert = pymupdf4llm.to_markdown("./train/입영일자 연기.pdf")

Processing ./train/입영일자 연기.pdf...
[                                        ] (0/6=====[======                                  ] (1/6======[=============                           ] (2/======[====================                    ] (3/6=====[==========================              ] (4/6======[=================================       ] (5/======[========================================] (6/6]


'''
'# 개요(총괄)\n\n\n## 총괄\n\n|총괄|Col2|Col3|Col4|Col5|Col6|Col7|\n|---|---|---|---|---|---|---|\n|18세|19세||20세 22세 24세 26세 27세 28세 30세 35세 37세 45세||||\n|병역 준비역 편입 · 안내|병역 판정 검사 실시|신체 등급 1급 ~ 4급|재학입영연기||||\n||||6년제 의과대학, 2년제 4년제 대학, 치과대학 등, 대학 대학 대학원 대학원 (4학기) (4학기 초과)||||\n||||병역의무이행|||기피자 · 미귀국 자|\n||||현역병 육군/해병(18개월), 해군(20개월), 공군(21개월) 상근예비역(18개월) 사회복무요원(21개월) 대체복무요원(36개월) 산업기능요원 현역입영대상자(34개월), 사회복무요원소집대상자(23개월) 전문연구요원(3년)||||\n|||||예비군8년|||\n|||5급|전 시 근 로 역 · 전 시 근 로 소 집 대상||||\n|||6급|병 역 면 제||||\n|||7급|재 신 체 검 사 2 년 이 내 확 정 처 분||||\n\n\n
'''

In [48]:
convert

'# 입영일자 연기\n\n[입영신청 절차 및 복무기간](https://mma.go.kr/contents.do?mc=mma0000728) [입영신청 공지사항](https://mma.go.kr/board/boardList.do?gesipan_id=507&mc=mma0003311) [입영안내 길잡이](https://mma.go.kr/contents.do?mc=mma0000739)\n입영일자 [연기](https://mma.go.kr/contents.do?mc=mma0000729)\n\n병역판정검사를 받은 해 입영희망 상근 예비역제도 안내 귀가자 처리안내\n\n## 입영일자 연기\n\n### 개요\n\n현역병 입영 대상자로서 질병 등 부득이한 사유로 지정된 입영일자에 입영할 수 없는 사람이 병역이행일 연기신청서를 제\n\n출하면 이를 심사하여 입영일자를 연기하는 제도\n\n### 신청대상\n\n현역병입영 대상자로서 아래 연기사유에 해당되는 사람. 단, 입영일자/입영부대를 본인이 선택한 사람은 입영일자 연기\n(질병, 직계 존·비속 간호, 천재·지변, 행방불명, 각군지원 사유자는 제외) 처리 제한\n\n### 신청시기\n\n입영일 5일전까지 신청이 가능\n\n### 신청기관\n\n지방병무청 현역입영 부서(현역입영과, 병역판정입영과, 병역관리과)\n\n우편으로 제출시는 병무청 「병무민원포털-신청서/구비서류 」 병역이행일 연기신청서 를 다운받아 해\n\n[바로가기](https://mwpt.mma.go.kr/caisBMHS/index_mwps.jsp?menuNo=22210&mwgsgeul_yhcd=04&mwgsgeul_no=95)\n\n당란 기재한 후 입영통지서를 받은 지방병무청(지청)으로 송부\n\n### 입영일자 연기사유, 연기기간 및 구비서류\n\n병역이행일 연기신청서와 함께 아래 사유별 구비서류를 첨부하여 제출하면 통산 2년의 범위안에서 연기\n\n|입영신청 절차 및 복무기간|입영신청 공지사항|Col3|입영일자연기|Col5|입영안내 길잡이|Col7|\n|---|---|

In [42]:
graph = "|Col2|Col3|Col4|Col5|Col6|Col7|\n|---|---|---|---|---|---|---|\n|18세|19세||20세 22세 24세 26세 27세 28세 30세 35세 37세 45세||||\n|병역 준비역 편입 · 안내|병역 판정 검사 실시|신체 등급 1급 ~ 4급|재학입영연기||||\n||||6년제 의과대학, 2년제 4년제 대학, 치과대학 등, 대학 대학 대학원 대학원 (4학기) (4학기 초과)||||\n||||병역의무이행|||기피자 · 미귀국 자|\n||||현역병 육군/해병(18개월), 해군(20개월), 공군(21개월) 상근예비역(18개월) 사회복무요원(21개월) 대체복무요원(36개월) 산업기능요원 현역입영대상자(34개월), 사회복무요원소집대상자(23개월) 전문연구요원(3년)||||\n||"

In [43]:
graph2= "|Col2|Col3|Col4|Col5|Col6|Col7|\n|---|---|---|---|---|---|---|\n|18세|19세||20세 22세 24세 26세 27세 28세 30세 35세 37세 45세||||\n|병역 준비역 편입 · 안내|병역 판정 검사 실시|신체 등급 1급 ~ 4급|재학입영연기||||\n||||6년제 의과대학, 2년제 4년제 대학, 치과대학 등, 대학 대학 대학원 대학원 (4학기) (4학기 초과)||||\n||||병역의무이행|||기피자 · 미귀국 자|\n||||현역병 육군/해병(18개월), 해군(20개월), 공군(21개월) 상근예비역(18개월) 사회복무요원(21개월) 대체복무요원(36개월) 산업기능요원 현역입영대상자(34개월), 사회복무요원소집대상자(23개월) 전문연구요원(3년)||||\n|||||예비군8년|||\n|||5급|전 시 근 로 역 · 전 시 근 로 소 집 대상||||\n|||6급|병 역 면 제||||\n|||7급|재 신 체 검 사 2 년 이 내 확 정 처 분||||"

In [62]:
messages = [
    SystemMessage(content = "You are EXAONE model from LG AI Research, a helpful assistant."),
    HumanMessage(
        content = f"{convert}, 이 글을 토대로만 질문에 대답해줘. 질문: 나 벤처캐피탈로부터 투자를 받아서 입영을 연기하고 싶은데 가능할까"
        # content = f"{graph2} 이 표를 너가 이해하고 학습할 수 있는 형태로 바꿔줘. 표 형식이 아닌 개조식으로 나타내줘. 어느 나이에 어떤 과정을 거쳐야하는지 알려줘"
    )
]
ai_msg = chat_model.invoke(messages)
ai_msg.content.split('[|assistant|]')[1]

'네, 가능합니다. 벤처캐피탈로부터 투자를 받은 경우, 입영일자 연기가 허용되는 사유 중 하나로 분류될 수 있습니다. 주어진 정보에 따르면, 벤처캐피탈로부터 투자를 받은 사람은 **벤처캐피탈 투자 사실 확인서**와 같은 증빙서류를 제출하면 입영일자를 연기할 수 있습니다. 하지만, 구체적인 연기 기간은 일반적인 사유들에 비해 제한적일 수 있으며, 정확한 절차와 가능 기간은 지방병무청에 문의하시는 것이 가장 좋습니다. 신청 시 관련 증빙서류를 준비하여 제출하시면 됩니다.'